In [1]:
# load necessary libraries
import pandas as pd 
import glob
import os
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_score, LeaveOneOut
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load all data previously retrieved from different sources
tg_2020 = pd.read_csv("tg_data/tg_2020.csv") # telefram 2020 and 2022
tg_2022 = pd.read_csv("tg_data/tg_2022.csv")
gt_2020 = pd.read_csv("gt_data/gt_2020.csv") # google trends 2020 and 2022
gt_2022 = pd.read_csv("gt_data/gt_2022.csv")
news_2020 = pd.read_csv("news_data/news_2020.csv") # news outlets 2020 and 2022
news_2022 = pd.read_csv("news_data/news_2022.csv")  
def load_yt_data_by_year(year):
    csv_path = f"yt_data/yt_{year}/*.csv"
    csv_files = glob.glob(csv_path)
    dataframes = []
    for file in csv_files:
        df = pd.read_csv(file)
        df['source_file'] = os.path.basename(file)
        dataframes.append(df)
    return pd.concat(dataframes, ignore_index=True)
yt_2020 = load_yt_data_by_year(2020) # youtube 2020 and 2022
yt_2022 = load_yt_data_by_year(2022)
off_2020 = pd.read_csv("official_data/stats_2020.csv") # official data 2020 and 2022
off_2022 = pd.read_csv("official_data/stats_2022.csv")

In [3]:
tg_2020.head()

,state,dimension,count,percentage,total_posts
0,Guanajuato,INCOME,42,20.487805,205
1,Guanajuato,ACCESS TO HEALTH SERVICES,58,28.292683,205
2,Guanajuato,EDUCATIONAL_LAG,12,5.853659,205
3,Guanajuato,ACCESS TO SOCIAL SECURITY,56,27.317073,205
4,Guanajuato,HOUSING,11,5.365854,205


In [4]:
gt_2020.head()

,state,FOVISSSTE,PromoDescuentos,agua potable,apoyo Infonavit,banco de alimentos,becas,centro de salud,conflictos,crisis,...,tianguis,tiendeo,violencia,access_to_food,access_to_health_services,access_to_social_security,educational_lag,housing,income,social_cohesion
0,MX-AGU,5.08,1.00,1.33,3.42,0.00,27.08,4.00,26.92,24.75,...,0.83,0.00,9.50,0.92,12.75,35.88,15.75,3.28,16.08,19.42
1,MX-BCN,37.67,7.50,18.00,71.25,0.08,14.33,3.83,34.92,25.25,...,25.50,3.50,6.58,9.14,10.95,39.42,9.29,42.31,18.22,21.03
2,MX-BCS,34.67,0.75,11.75,13.83,0.00,6.83,0.92,3.75,2.92,...,1.00,0.00,2.67,0.88,3.50,5.66,3.83,20.08,1.72,3.06
3,MX-CAM,2.08,54.33,1.17,0.08,0.00,5.75,0.08,31.42,4.17,...,26.25,2.08,2.17,27.55,2.54,2.96,3.42,1.11,2.39,20.36
4,MX-CHP,10.42,5.25,2.67,0.42,0.08,7.42,0.25,16.25,7.67,...,4.67,0.25,2.58,2.56,3.21,8.17,4.25,4.50,4.44,10.75


In [5]:
yt_2020.head()

,state,dimension,avg_sentiment,mentions_count,percentage_of_total,videos_analyzed,comments_analyzed,source_file
0,Morelos,Income,-0.426325,1717,34.554236,300,4850,morelos.csv
1,Morelos,Access To Health Services,-0.497436,585,11.772993,300,4850,morelos.csv
2,Morelos,Educational Lag,-0.245375,1027,20.668142,300,4850,morelos.csv
3,Morelos,Access To Social Security,-0.265060,83,1.670356,300,4850,morelos.csv
4,Morelos,Housing,-0.430380,237,4.769571,300,4850,morelos.csv


In [6]:
news_2020.head()

,Unnamed: 0,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7
0,Chiapas,0.049664,0.428592,0.050325,0.066440,0.168499,0.105813,0.068651,0.060388
1,Queretaro,0.219801,0.296963,0.051571,0.099180,0.111831,0.090218,0.040383,0.089356
2,Baja California Sur,0.038544,0.295954,0.274110,0.027095,0.112471,0.119586,0.074339,0.057354
3,Guanajuato,0.036158,0.331253,0.006775,0.050556,0.115823,0.317956,0.043033,0.097156
4,Hidalgo,0.026875,0.034518,0.042831,0.264087,0.097462,0.052220,0.449243,0.031114


In [7]:
off_2020.head()

,state,educational lag,access to health services,access to social security,housing1,housing2,income,access to food
0,Aguascalientes,15.7,20.2,36.0,4.0,2.1,38.7,18.2
1,Baja California,16.1,24.0,38.9,6.8,5.0,30.4,13.4
2,Baja California Sur,14.5,17.4,32.7,11.4,9.4,35.9,23.4
3,Campeche,19.2,21.0,54.9,13.5,33.4,57.7,26.1
4,Coahuila de Zaragoza,14.3,21.6,28.2,2.8,3.2,40.2,15.5


## 1. From CONEVAL's statistics, average housing1 and housing2 to create only one measure for this dimension and drop unnecessary columns

In [8]:
off_2020.columns = off_2020.columns.str.strip().str.replace(' ', '_')
off_2022.columns = off_2022.columns.str.strip().str.replace(' ', '_')

In [9]:
off_2020['housing'] = (off_2020['housing1'] + off_2020['housing2']) / 2
off_2022['housing'] = (off_2022['housing1'] + off_2022['housing2']) / 2

off_2020.drop(columns=['housing1', 'housing2'], inplace=True)
off_2022.drop(columns=['housing1', 'housing2'], inplace=True)

In [10]:
off_2020.head()

,state,educational_lag,access_to_health_services,access_to_social_security,income,access_to_food,housing
0,Aguascalientes,15.7,20.2,36.0,38.7,18.2,3.05
1,Baja California,16.1,24.0,38.9,30.4,13.4,5.90
2,Baja California Sur,14.5,17.4,32.7,35.9,23.4,10.40
3,Campeche,19.2,21.0,54.9,57.7,26.1,23.45
4,Coahuila de Zaragoza,14.3,21.6,28.2,40.2,15.5,3.00


In [11]:
off_2022.head()

,year,state,educational_lag,access_to_health_services,access_to_social_security,access_to_food,income,ext_pov,housing
0,2022,Aguascalientes,16.5,26.2,34.7,15.6,33.0,6.1,3.60
1,2022,Baja California,15.5,28.4,34.7,10.0,18.8,2.8,5.10
2,2022,Baja California Sur,14.2,17.3,29.9,14.5,17.9,2.8,9.25
3,2022,Campeche,20.3,34.6,54.7,22.0,51.0,15.9,24.90
4,2022,Coahuila de Zaragoza,13.5,19.7,24.4,13.2,28.8,5.9,3.40


In [12]:
off_2022.drop(columns=['ext_pov', 'year'], inplace=True)

In [13]:
off_2020.rename(columns={'educational_lag': 'educ_target', 'access_to_health_services': 'health_target', 
                         'access_to_social_security': 'social_target', 'access_to_food': 'food_target',
                          'income': 'income_target', 'housing': 'housing_target'}, inplace=True)

off_2022.rename(columns={'educational_lag': 'educ_target', 'access_to_health_services': 'health_target', 
                         'access_to_social_security': 'social_target', 'access_to_food': 'food_target',
                          'income': 'income_target', 'housing': 'housing_target'}, inplace=True)

In [14]:
off_2020.head()

,state,educ_target,health_target,social_target,income_target,food_target,housing_target
0,Aguascalientes,15.7,20.2,36.0,38.7,18.2,3.05
1,Baja California,16.1,24.0,38.9,30.4,13.4,5.90
2,Baja California Sur,14.5,17.4,32.7,35.9,23.4,10.40
3,Campeche,19.2,21.0,54.9,57.7,26.1,23.45
4,Coahuila de Zaragoza,14.3,21.6,28.2,40.2,15.5,3.00


## 2. Clean Google Trends data by removing unnecesary columns and renaming columns for consistency 

In [15]:
gt_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   state                      32 non-null     object 
 1   FOVISSSTE                  32 non-null     float64
 2   PromoDescuentos            32 non-null     float64
 3   agua potable               32 non-null     float64
 4   apoyo Infonavit            32 non-null     float64
 5   banco de alimentos         32 non-null     float64
 6   becas                      32 non-null     float64
 7   centro de salud            32 non-null     float64
 8   conflictos                 32 non-null     float64
 9   crisis                     32 non-null     float64
 10  desempleo                  32 non-null     float64
 11  discriminación            32 non-null     float64
 12  enfermedad                 32 non-null     float64
 13  escuela secundaria         32 non-null     float64
 

In [16]:
POVERTY_DIMENSIONS = {
    'income': ['crisis', 'desempleo', 'pobreza'],
    'access_to_health_services': ['enfermedad', 'centro de salud'],
    'educational_lag': ['becas', 'escuela secundaria'],
    'access_to_social_security': ['pensiones', 'seguro social'],
    'access_to_food': ['banco de alimentos', 'tianguis', 'tiendeo', 'PromoDescuentos'],
    'housing': ['apoyo Infonavit', 'agua potable', 'FOVISSSTE'],
    'social_cohesion': ['violencia', 'conflictos', 'discriminación']}

In [17]:
# remove all columns exceot the state and poverty dimensions
gt_2020 = gt_2020[['state'] + list(POVERTY_DIMENSIONS.keys())]

gt_2022 = gt_2022[['state'] + list(POVERTY_DIMENSIONS.keys())]

In [18]:
# rename states with the full name 
state_map = {
    "MX-AGU": "Aguascalientes", "MX-BCN": "Baja California", "MX-BCS": "Baja California Sur",
    "MX-CAM": "Campeche", "MX-CHP": "Chiapas", "MX-CHH": "Chihuahua", "MX-COA": "Coahuila",
    "MX-COL": "Colima", "MX-DUR": "Durango", "MX-GUA": "Guanajuato", "MX-GRO": "Guerrero",
    "MX-HID": "Hidalgo", "MX-JAL": "Jalisco", "MX-DIF": "Ciudad de México", "MX-MIC": "Michoacán",
    "MX-MOR": "Morelos", "MX-NAY": "Nayarit", "MX-NLE": "Nuevo León", "MX-OAX": "Oaxaca",
    "MX-PUE": "Puebla", "MX-QUE": "Querétaro", "MX-ROO": "Quintana Roo", "MX-SLP": "San Luis Potosí",
    "MX-SIN": "Sinaloa", "MX-SON": "Sonora", "MX-MEX": "Estado de México", "MX-TAB": "Tabasco",
    "MX-TAM": "Tamaulipas", "MX-TLA": "Tlaxcala", "MX-VER": "Veracruz", "MX-YUC": "Yucatán", "MX-ZAC": "Zacatecas"}

gt_2020['state'] = gt_2020['state'].map(state_map)
gt_2022['state'] = gt_2022['state'].map(state_map)

In [19]:
gt_2020.head()

,state,income,access_to_health_services,educational_lag,access_to_social_security,access_to_food,housing,social_cohesion
0,Aguascalientes,16.08,12.75,15.75,35.88,0.92,3.28,19.42
1,Baja California,18.22,10.95,9.29,39.42,9.14,42.31,21.03
2,Baja California Sur,1.72,3.50,3.83,5.66,0.88,20.08,3.06
3,Campeche,2.39,2.54,3.42,2.96,27.55,1.11,20.36
4,Chiapas,4.44,3.21,4.25,8.17,2.56,4.50,10.75


In [20]:
gt_2020.rename(columns={'income': 'income_gt', 'access_to_health_services': 'health_gt', 'eduational_lag': 'education_gt',
                        'access_to_social_security': 'social_gt', 'access_to_food': 'food_gt', 'housing': 'housing_gt',
                        'social_cohesion': 'cohesion_gt'}, inplace=True)

gt_2022.rename(columns={'income': 'income_gt', 'access_to_health_services': 'health_gt', 'eduational_lag': 'education_gt',
                        'access_to_social_security': 'social_gt', 'access_to_food': 'food_gt', 'housing': 'housing_gt',
                        'social_cohesion': 'cohesion_gt'}, inplace=True)

In [21]:
gt_2020.head()

,state,income_gt,health_gt,educational_lag,social_gt,food_gt,housing_gt,cohesion_gt
0,Aguascalientes,16.08,12.75,15.75,35.88,0.92,3.28,19.42
1,Baja California,18.22,10.95,9.29,39.42,9.14,42.31,21.03
2,Baja California Sur,1.72,3.50,3.83,5.66,0.88,20.08,3.06
3,Campeche,2.39,2.54,3.42,2.96,27.55,1.11,20.36
4,Chiapas,4.44,3.21,4.25,8.17,2.56,4.50,10.75


## 3. Clean Youtube by computing the percentage of mentions for each poverty dimension (from the raw count), reshaping and removing unnecessary columns

In [6]:
yt_2020.head(8)

,state,dimension,avg_sentiment,mentions_count,percentage_of_total,videos_analyzed,comments_analyzed,source_file
0,Morelos,Income,-0.426325,1717,34.554236,300,4850,morelos.csv
1,Morelos,Access To Health Services,-0.497436,585,11.772993,300,4850,morelos.csv
2,Morelos,Educational Lag,-0.245375,1027,20.668142,300,4850,morelos.csv
3,Morelos,Access To Social Security,-0.265060,83,1.670356,300,4850,morelos.csv
4,Morelos,Housing,-0.430380,237,4.769571,300,4850,morelos.csv
5,Morelos,Access To Food,-0.520566,389,7.828537,300,4850,morelos.csv
6,Morelos,Social Cohesion,-0.163991,436,8.774401,300,4850,morelos.csv
7,Morelos,Other,-0.685859,495,9.961763,300,4850,morelos.csv


In [7]:
yt_2020.drop(columns=['source_file'], inplace=True) 
yt_2022.drop(columns=['source_file'], inplace=True)

yt_2020.drop(columns=['videos_analyzed'], inplace=True)
yt_2022.drop(columns=['videos_analyzed'], inplace=True)

In [8]:
yt_2020.rename(columns={'percentage_of_total': 'pct_yt'}, inplace=True)
yt_2022.rename(columns={'percentage_of_total': 'pct_yt'}, inplace=True)

In [9]:
# drop the 'mentions_count' and 'comments_analyzed' columns since now are redunant
yt_2020.drop(columns=['mentions_count', 'comments_analyzed'], inplace=True)
yt_2022.drop(columns=['mentions_count', 'comments_analyzed'], inplace=True)

In [11]:
yt_2020.head()

,state,dimension,avg_sentiment,pct_yt
0,Morelos,Income,-0.426325,34.554236
1,Morelos,Access To Health Services,-0.497436,11.772993
2,Morelos,Educational Lag,-0.245375,20.668142
3,Morelos,Access To Social Security,-0.265060,1.670356
4,Morelos,Housing,-0.430380,4.769571


In [12]:
yt_2020 = yt_2020.pivot(index='state', columns='dimension', values=['avg_sentiment', 'pct_yt'])
yt_2020.columns = [f'{dim.lower().replace(" ", "_")}_{metric}' for metric, dim in yt_2020.columns]
yt_2020 = yt_2020.reset_index()

yt_2022 = yt_2022.pivot(index='state', columns='dimension', values=['avg_sentiment', 'pct_yt'])
yt_2022.columns = [f'{dim.lower().replace(" ", "_")}_{metric}' for metric, dim in yt_2022.columns]
yt_2022 = yt_2022.reset_index()

In [13]:
yt_2020.head()

,state,access_to_food_avg_sentiment,access_to_health_services_avg_sentiment,access_to_social_security_avg_sentiment,educational_lag_avg_sentiment,housing_avg_sentiment,income_avg_sentiment,other_avg_sentiment,social_cohesion_avg_sentiment,access_to_food_pct_yt,access_to_health_services_pct_yt,access_to_social_security_pct_yt,educational_lag_pct_yt,housing_pct_yt,income_pct_yt,other_pct_yt,social_cohesion_pct_yt
0,Aguascalientes,-0.299735,-0.592422,-0.638889,-0.234419,-0.255694,-0.441204,-0.568582,-0.162062,7.425645,13.905850,1.418160,20.858775,4.756746,31.741186,10.626354,9.267284
1,Baja California,-0.456311,-0.484561,-0.343434,-0.150926,-0.398467,-0.436123,-0.572650,-0.127451,8.468073,11.537408,2.713072,14.798575,7.152645,37.325295,9.619074,8.385859
2,Baja California Sur,-0.506224,-0.069231,-0.183333,0.225490,-0.455696,-0.224547,-0.565217,0.223629,10.574814,8.556384,2.632734,17.902589,10.399298,31.461167,8.073717,10.399298
3,Campeche,-0.373233,-0.485835,-0.369492,-0.113962,-0.530670,-0.347170,-0.513535,0.092805,8.510468,10.475656,2.366247,21.256116,5.622844,31.884174,10.074597,9.809898
4,Chiapas,-0.386760,-0.446627,-0.351786,-0.160019,-0.523504,-0.294440,-0.616719,-0.155708,5.625245,13.220306,2.744022,20.638965,4.586437,33.134065,9.319875,10.731086


## 4. News data: name topics retrieved from LDA analysis

In [3]:
news_2020.head()

,Unnamed: 0,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7
0,Chiapas,0.049664,0.428592,0.050325,0.066440,0.168499,0.105813,0.068651,0.060388
1,Queretaro,0.219801,0.296963,0.051571,0.099180,0.111831,0.090218,0.040383,0.089356
2,Baja California Sur,0.038544,0.295954,0.274110,0.027095,0.112471,0.119586,0.074339,0.057354
3,Guanajuato,0.036158,0.331253,0.006775,0.050556,0.115823,0.317956,0.043033,0.097156
4,Hidalgo,0.026875,0.034518,0.042831,0.264087,0.097462,0.052220,0.449243,0.031114


In [4]:
news_2020.rename(columns={'Unnamed: 0': 'state', 'Topic_0': 'economy', 'Topic_1': 'women_family',
                         'Topic_2': 'employment', 'Topic_3': 'public_services', 'Topic_4': 'security', 'Topic_5': 'health',
                          'Topic_6': 'breaking_news',  'Topic_7': 'politics'}, inplace=True)

news_2022.rename(columns={'Unnamed: 0': 'state', 'Topic_0': 'economy', 'Topic_1': 'women_family',
                         'Topic_2': 'employment', 'Topic_3': 'public_services', 'Topic_4': 'security', 'Topic_5': 'health',
                          'Topic_6': 'breaking_news',  'Topic_7': 'politics'}, inplace=True)

In [5]:
news_2022.head()

,state,economy,women_family,employment,public_services,security,health,breaking_news,politics
0,Chiapas,0.037970,0.348579,0.071140,0.047274,0.167165,0.128491,0.109026,0.088620
1,Queretaro,0.253777,0.295403,0.048283,0.117081,0.107642,0.073159,0.037829,0.065943
2,Baja California Sur,0.049376,0.266573,0.264362,0.045150,0.109812,0.142678,0.039559,0.081724
3,Guanajuato,0.061663,0.242712,0.009676,0.038500,0.139402,0.370912,0.019274,0.116210
4,Hidalgo,0.038842,0.037922,0.047457,0.223881,0.119813,0.075366,0.406752,0.047702


## 5. Clean Telegram by reshaping and removing unnecessary columns

In [32]:
tg_2020.head()

,state,dimension,count,percentage,total_posts
0,Guanajuato,INCOME,42,20.487805,205
1,Guanajuato,ACCESS TO HEALTH SERVICES,58,28.292683,205
2,Guanajuato,EDUCATIONAL_LAG,12,5.853659,205
3,Guanajuato,ACCESS TO SOCIAL SECURITY,56,27.317073,205
4,Guanajuato,HOUSING,11,5.365854,205


In [33]:
tg_2020.drop(columns=['count', 'total_posts'], inplace=True)
tg_2022.drop(columns=['count', 'total_posts'], inplace=True)

tg_2020.rename(columns={'percentage': 'pct_tg'}, inplace=True)
tg_2022.rename(columns={'percentage': 'pct_tg'}, inplace=True)

In [34]:
tg_2020 = tg_2020.pivot(index='state', columns='dimension', values=['pct_tg'])
tg_2020.columns = [f'{dim.lower().replace(" ", "_")}_{metric}' for metric, dim in tg_2020.columns]
tg_2020 = tg_2020.reset_index()

tg_2022 = tg_2022.pivot(index='state', columns='dimension', values=['pct_tg'])
tg_2022.columns = [f'{dim.lower().replace(" ", "_")}_{metric}' for metric, dim in tg_2022.columns]
tg_2022 = tg_2022.reset_index()

In [35]:
tg_2020.head()

,state,access_to_food_pct_tg,access_to_health_services_pct_tg,access_to_social_security_pct_tg,educational_lag_pct_tg,housing_pct_tg,income_pct_tg,other_pct_tg,social_cohesion_pct_tg
0,Aguascalientes,0.000000,26.315789,36.842105,10.526316,0.00000,21.052632,5.263158,0.00000
1,Baja California,0.000000,37.142857,25.714286,0.000000,0.00000,22.857143,14.285714,0.00000
2,Baja California Sur,0.000000,57.142857,28.571429,0.000000,0.00000,0.000000,14.285714,0.00000
3,Campeche,4.444444,33.333333,24.444444,13.333333,0.00000,8.888889,15.555556,0.00000
4,Chiapas,6.382979,46.808511,20.212766,11.702128,1.06383,9.574468,3.191489,1.06383


In [36]:
# save all clean dataframes to csv files
tg_2020.to_csv("clean_data/tg_2020.csv", index=False)
tg_2022.to_csv("clean_data/tg_2022.csv", index=False)
gt_2020.to_csv("clean_data/gt_2020.csv", index=False)
gt_2022.to_csv("clean_data/gt_2022.csv", index=False)
yt_2020.to_csv("clean_data/yt_2020.csv", index=False)
yt_2022.to_csv("clean_data/yt_2022.csv", index=False)
news_2020.to_csv("clean_data/news_2020.csv", index=False)
news_2022.to_csv("clean_data/news_2022.csv", index=False)
off_2020.to_csv("clean_data/off_2020.csv", index=False)
off_2022.to_csv("clean_data/off_2022.csv", index=False)